# HVNM Small-Amplitude Oscillatory Shear (SAOS)

**Hybrid Vitrimer Nanocomposite Model — Three Maxwell modes + reinforced plateau**

## Learning Objectives

- Predict SAOS moduli for a 4-subnetwork vitrimer nanocomposite
- Identify G_P*X(phi) from the low-frequency plateau
- Observe the dual factor-of-2: τ_E_eff = 1/(2·k_BER_0) and τ_I_eff = 1/(2·k_BER_0)
- Understand phi-dependent reinforcement and three loss peaks

## Estimated Runtime

- ~30s (analytical)

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import HVNMLocal

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings("ignore", category=FutureWarning)

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory

The HVNM SAOS moduli are a superposition of three Maxwell modes plus a reinforced permanent plateau:

$$G'(\omega) = G_P X(\phi) + \frac{G_E \omega^2 \tau_{E,eff}^2}{1 + \omega^2 \tau_{E,eff}^2} + \frac{G_I \omega^2 \tau_{I,eff}^2}{1 + \omega^2 \tau_{I,eff}^2} + \frac{G_D \omega^2 \tau_D^2}{1 + \omega^2 \tau_D^2}$$

$$G''(\omega) = \frac{G_E \omega \tau_{E,eff}}{1 + \omega^2 \tau_{E,eff}^2} + \frac{G_I \omega \tau_{I,eff}}{1 + \omega^2 \tau_{I,eff}^2} + \frac{G_D \omega \tau_D}{1 + \omega^2 \tau_D^2}$$

where:
- $\tau_{E,eff} = 1/(2 k_{BER,0})$ (factor-of-2 for matrix exchange)
- $\tau_{I,eff} = 1/(2 k_{BER,0})$ (factor-of-2 for interphase exchange, same rate)
- $\tau_D = 1/k_d^D$ (dissociative relaxation)
- $X(\phi) = (1 + 2.5\phi + 14.1\phi^2)$ (Einstein-Batchelor reinforcement)

**Key features:**
- **Low-ω:** $G'(\omega \to 0) = G_P X(\phi)$ (reinforced permanent plateau)
- **High-ω:** $G'(\omega \to \infty) = G_P X(\phi) + G_E + G_I + G_D$ (total modulus)
- **Three loss peaks** in $G''$: one at $\omega \sim 1/\tau_{E,eff}$, one at $\omega \sim 1/\tau_{I,eff}$, one at $\omega \sim 1/\tau_D$

## 3. Model Setup

In [ ]:
# Create full HVNM model
model = HVNMLocal()
model.parameters.set_value("G_P", 5000.0)
model.parameters.set_value("G_E", 3000.0)
model.parameters.set_value("G_I", 2000.0)
model.parameters.set_value("G_D", 1000.0)
model.parameters.set_value("phi", 0.1)
model.parameters.set_value("beta_I", 3.0)
model.parameters.set_value("nu_0", 1e10)
model.parameters.set_value("E_a", 80e3)
model.parameters.set_value("V_act", 1e-5)
model.parameters.set_value("T", 350.0)
model.parameters.set_value("k_d_D", 10.0)

# Check effective relaxation times
tau_E = model.get_vitrimer_relaxation_time()
tau_I = model.get_interphase_relaxation_time()
tau_D = 1.0 / model.k_d_D
k_BER_0 = model.compute_ber_rate_at_equilibrium()
X = model.compute_reinforcement_factor()
regime = model.classify_vitrimer_regime()

print(f"k_BER_0 = {k_BER_0:.4f} 1/s")
print(f"τ_E_eff = 1/(2·k_BER_0) = {tau_E:.4f} s")
print(f"τ_I_eff = 1/(2·k_BER_0) = {tau_I:.4f} s")
print(f"τ_D     = 1/k_d_D      = {tau_D:.4f} s")
print(f"X(φ={model.phi}) = {X:.3f}")
print(f"G_P*X = {model.G_P * X:.0f} Pa")
print(f"Regime: {regime}")
print(f"Limiting case: {model.get_limiting_case()}")
print(f"Network fractions: {model.get_network_fractions()}")

## 4. SAOS Prediction

In [ ]:
omega = np.logspace(-3, 3, 200)
G_prime, G_double_prime = model.predict_saos(omega)

X = model.compute_reinforcement_factor()
G_tot = model.G_P * X + model.G_E + model.G_I + model.G_D

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# G', G'' vs omega
ax1.loglog(omega, G_prime, "k-", lw=2, label="G'")
ax1.loglog(omega, G_double_prime, "r-", lw=2, label="G''")
ax1.axhline(model.G_P * X, color="blue", ls=":", alpha=0.5, label=f"G_P*X = {model.G_P * X:.0f}")
ax1.axhline(G_tot, color="green", ls=":", alpha=0.5, label=f"G_tot = {G_tot:.0f}")
ax1.axvline(1/tau_E, color="gray", ls="--", alpha=0.4, label=f"1/τ_E_eff = {1/tau_E:.2f}")
ax1.axvline(1/tau_I, color="purple", ls="--", alpha=0.4, label=f"1/τ_I_eff = {1/tau_I:.2f}")
ax1.axvline(1/tau_D, color="orange", ls="--", alpha=0.4, label=f"1/τ_D = {1/tau_D:.1f}")
ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G', G'' [Pa]")
ax1.set_title("HVNM SAOS: Three Maxwell Modes + Reinforced Plateau")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

# tan(delta)
tan_delta = G_double_prime / G_prime
ax2.semilogx(omega, tan_delta, "b-", lw=2)
ax2.set_xlabel("ω [rad/s]")
ax2.set_ylabel("tan(δ)")
ax2.set_title("Loss Tangent")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Dual Factor-of-2 Demonstration

In [ ]:
# Compare E-network and I-network contributions: both with factor-of-2
tau_E_naive = 1.0 / k_BER_0  # Without factor-of-2
tau_E_correct = tau_E          # With factor-of-2
tau_I_correct = tau_I          # With factor-of-2 (same rate)

G_E_prime_naive = model.G_E * (omega * tau_E_naive)**2 / (1 + (omega * tau_E_naive)**2)
G_E_prime_correct = model.G_E * (omega * tau_E_correct)**2 / (1 + (omega * tau_E_correct)**2)
G_I_prime_correct = model.G_I * (omega * tau_I_correct)**2 / (1 + (omega * tau_I_correct)**2)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(omega, G_prime - model.G_P * X, "k-", lw=2, label="HVNM (full, minus G_P*X)")
ax.loglog(omega, G_E_prime_correct + G_I_prime_correct, "g--", lw=2, 
          label=f"E+I modes: τ_eff = {tau_E_correct:.3f}s (correct, both)")
ax.loglog(omega, G_E_prime_naive, "r:", lw=2, 
          label=f"E-mode: τ_E = {tau_E_naive:.3f}s (wrong, no factor-of-2)")
ax.set_xlabel("ω [rad/s]")
ax.set_ylabel("G' - G_P*X [Pa]")
ax.set_title("Dual Factor-of-2: τ_E_eff = τ_I_eff = 1/(2·k_BER_0)")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Nanoparticle Volume Fraction Sweep

In [ ]:
# Compare SAOS for different nanoparticle loadings
phi_values = [0.0, 0.05, 0.1, 0.15, 0.2]
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(phi_values)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for phi_val, color in zip(phi_values, colors):
    model.parameters.set_value("phi", float(phi_val))
    G_p, G_dp = model.predict_saos(omega)
    X_phi = model.compute_reinforcement_factor()
    ax1.loglog(omega, G_p, "-", color=color, lw=2, 
               label=f"φ = {phi_val:.2f}, X = {X_phi:.2f}")
    ax1.loglog(omega, G_dp, "--", color=color, lw=1, alpha=0.6)

ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G' (solid), G'' (dashed) [Pa]")
ax1.set_title("Nanoparticle Volume Fraction Sweep")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

# Reinforcement factor vs phi
phi_range = np.linspace(0, 0.3, 100)
X_range = np.array([1 + 2.5*p + 14.1*p**2 for p in phi_range])
ax2.plot(phi_range, X_range, "b-", lw=2)
ax2.scatter(phi_values, [1 + 2.5*p + 14.1*p**2 for p in phi_values], 
            c='red', s=50, zorder=5)
ax2.set_xlabel("φ")
ax2.set_ylabel("X(φ)")
ax2.set_title("Einstein-Batchelor Reinforcement")
ax2.grid(True, alpha=0.3)

# Reset to original value
model.parameters.set_value("phi", 0.1)

plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Temperature Sweep (Dual Arrhenius)

In [ ]:
temperatures = [300, 325, 350, 375, 400]
colors = plt.cm.RdYlBu(np.linspace(0.8, 0.2, len(temperatures)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for T_val, color in zip(temperatures, colors):
    model.parameters.set_value("T", float(T_val))
    G_p, G_dp = model.predict_saos(omega)
    ax1.loglog(omega, G_p, "-", color=color, lw=1.5, label=f"T = {T_val} K")
    ax1.loglog(omega, G_dp, "--", color=color, lw=1, alpha=0.7)

ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G' (solid), G'' (dashed) [Pa]")
ax1.set_title("Temperature Sweep")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

# Dual Arrhenius plot
model.parameters.set_value("T", 350.0)  # Reset
T_range = np.linspace(280, 450, 100)
arrhenius_data = model.arrhenius_plot_data_dual(T_range)
inv_T = arrhenius_data["inv_T"]
log_k_E = arrhenius_data["log_k_E"]
log_k_I = arrhenius_data["log_k_I"]

ax2.plot(inv_T, log_k_E, "b-", lw=2, label="k_BER_0 (E-network)")
ax2.plot(inv_T, log_k_I, "r--", lw=2, label="k_BER_0 (I-network)")
ax2.set_xlabel("1000/T [1/K]")
ax2.set_ylabel("log₁₀(k_BER_0) [1/s]")
ax2.set_title(f"Dual Arrhenius Plot (E_a = {model.E_a/1e3:.0f} kJ/mol)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 8. Limiting Cases Comparison

In [ ]:
# Compare limiting cases
full = HVNMLocal()
full.parameters.set_value("G_P", 5000.0)
full.parameters.set_value("G_E", 3000.0)
full.parameters.set_value("G_I", 2000.0)
full.parameters.set_value("G_D", 1000.0)
full.parameters.set_value("phi", 0.1)
full.parameters.set_value("beta_I", 3.0)
full.parameters.set_value("nu_0", 1e10)
full.parameters.set_value("E_a", 80e3)
full.parameters.set_value("V_act", 1e-5)
full.parameters.set_value("T", 350.0)
full.parameters.set_value("k_d_D", 10.0)

no_interphase = HVNMLocal.no_interphase(
    G_P=5000, G_E=3000, G_D=1000, phi=0.1,
    nu_0=1e10, E_a=80e3, V_act=1e-5, T=350.0, k_d_D=10.0
)

no_particles = HVNMLocal.no_particles(
    G_P=5000, G_E=3000, G_D=1000,
    nu_0=1e10, E_a=80e3, V_act=1e-5, T=350.0, k_d_D=10.0
)

fig, ax = plt.subplots(figsize=(8, 5))
for m, label, ls in [
    (full, "Full HVNM (4 networks)", "-"),
    (no_interphase, "No interphase (G_I=0)", "--"),
    (no_particles, "No particles (φ=0, G_I=0)", "-."),
]:
    G_p, G_dp = m.predict_saos(omega)
    ax.loglog(omega, G_p, ls, lw=2, label=f"G' {label}")

ax.set_xlabel("ω [rad/s]")
ax.set_ylabel("G' [Pa]")
ax.set_title("Limiting Cases")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## Key Takeaways

1. **G'(ω→0) = G_P*X(φ)**: The reinforced permanent network provides a phi-dependent low-frequency plateau
2. **Dual Factor-of-2**: Both τ_E_eff = 1/(2·k_BER_0) and τ_I_eff = 1/(2·k_BER_0) because μ^E, μ^E_nat AND μ^I, μ^I_nat relax toward each other
3. **Three loss peaks**: E-network at ω ~ 1/τ_E_eff, I-network at ω ~ 1/τ_I_eff, D-network at ω ~ 1/τ_D
4. **Einstein-Batchelor reinforcement**: X(φ) = 1 + 2.5φ + 14.1φ² increases plateau modulus with nanoparticle loading
5. **Dual Arrhenius**: Both matrix and interphase exchange rates follow k_BER_0(T) = ν₀·exp(-E_a/RT)

## Next Steps

- **Notebook 02**: Stress relaxation (tri-exponential + reinforced plateau)
- **Notebook 03**: Startup shear (TST stress overshoot with interphase contributions)
- **Notebook 04**: Creep compliance (reinforced glass + multi-mode retardation)
- **Notebook 05**: Flow curves (phi-dependent yield/plateau)